In [2]:
from sqlalchemy import create_engine

## Create engine

In [3]:
engine = create_engine('sqlite:///college.db', echo = True)

In [4]:
engine

Engine(sqlite:///college.db)

## MetaData for table creation

In [5]:
from sqlalchemy import MetaData
# Metadata contains definitions of tables and associated objects such as index, view, triggers, etc.
meta = MetaData()

In [6]:
# you can use the metadata obj to hold tables info
from sqlalchemy import Table, Column, Integer, String


In [7]:
# create a table object called students
students = Table(
    "students",
    meta, # the metadata obj to hold this table info
    Column('id', Integer, primary_key = True), 
    Column('name', String), 
    Column('lastname', String), 
)

In [8]:
# now you can actually use the metadata obj to execute
# create tables. It takes the engine as argument
# now this actually creates the database
meta.create_all(engine)

2021-04-21 20:06:09,600 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-04-21 20:06:09,601 INFO sqlalchemy.engine.base.Engine ()
2021-04-21 20:06:09,604 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-04-21 20:06:09,605 INFO sqlalchemy.engine.base.Engine ()
2021-04-21 20:06:09,607 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("students")
2021-04-21 20:06:09,608 INFO sqlalchemy.engine.base.Engine ()
2021-04-21 20:06:09,611 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE students (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	lastname VARCHAR, 
	PRIMARY KEY (id)
)


2021-04-21 20:06:09,612 INFO sqlalchemy.engine.base.Engine ()
2021-04-21 20:06:09,616 INFO sqlalchemy.engine.base.Engine COMMIT


## Use table methods to create expression

In [22]:
# create a sql query expression
ins = students.insert()
str(ins)

'INSERT INTO students (id, name, lastname) VALUES (:id, :name, :lastname)'

In [23]:
# bind and compile with values inserted
ins = ins.values(id=1, name="Bob", lastname="Smith")

In [25]:
# see the params
ins.compile().params

{'id': 1, 'name': 'Bob', 'lastname': 'Smith'}

In [26]:
str(ins)  # here you don't see it though

'INSERT INTO students (id, name, lastname) VALUES (:id, :name, :lastname)'

## Execute expression

In [27]:
# create connection
conn = engine.connect()

In [28]:
# we can use the expression or string query to execute
conn.execute(ins)

2021-04-21 20:16:25,490 INFO sqlalchemy.engine.base.Engine INSERT INTO students (id, name, lastname) VALUES (?, ?, ?)
2021-04-21 20:16:25,492 INFO sqlalchemy.engine.base.Engine (1, 'Bob', 'Smith')
2021-04-21 20:16:25,495 INFO sqlalchemy.engine.base.Engine COMMIT


In [29]:
# another 1
ins = students.insert().values(name = 'Ravi', lastname = 'Kapoor')
result = conn.execute(ins)


2021-04-21 20:17:26,743 INFO sqlalchemy.engine.base.Engine INSERT INTO students (name, lastname) VALUES (?, ?)
2021-04-21 20:17:26,744 INFO sqlalchemy.engine.base.Engine ('Ravi', 'Kapoor')
2021-04-21 20:17:26,746 INFO sqlalchemy.engine.base.Engine COMMIT


In [33]:
result = conn.execute("select * from students")

2021-04-21 20:18:53,471 INFO sqlalchemy.engine.base.Engine select * from students
2021-04-21 20:18:53,472 INFO sqlalchemy.engine.base.Engine ()


In [34]:
result.fetchall()

[(1, 'Bob', 'Smith'), (2, 'Ravi', 'Kapoor')]

## Textual SQL

In [36]:
from sqlalchemy import text
t = text("SELECT * FROM students")
result = conn.execute(t)

2021-04-21 20:22:26,749 INFO sqlalchemy.engine.base.Engine SELECT * FROM students
2021-04-21 20:22:26,751 INFO sqlalchemy.engine.base.Engine ()


In [37]:
result.fetchall()

[(1, 'Bob', 'Smith'), (2, 'Ravi', 'Kapoor')]

In [38]:
# You can format the query using text
s = text("select students.name, students.lastname from students where students.name between :x and :y")
conn.execute(s, x = 'A', y = 'L').fetchall()

2021-04-21 20:23:26,424 INFO sqlalchemy.engine.base.Engine select students.name, students.lastname from students where students.name between ? and ?
2021-04-21 20:23:26,430 INFO sqlalchemy.engine.base.Engine ('A', 'L')


[('Bob', 'Smith')]

In [40]:
from sqlalchemy.sql import select
s = select([text("students.name, students.lastname from students")]).where(text("students.name between :x and :y"))
conn.execute(s, x = 'A', y = 'L').fetchall()

2021-04-21 20:27:34,369 INFO sqlalchemy.engine.base.Engine SELECT students.name, students.lastname from students 
WHERE students.name between ? and ?
2021-04-21 20:27:34,370 INFO sqlalchemy.engine.base.Engine ('A', 'L')


[('Bob', 'Smith')]